## Sentiment140 ✨ KNN Classifier

### Dataset

Sentiment140 contains `1,600,000 tweets` extracted using the twitter api.  
The tweets have been annotated (`0 = negative, 4 = positive`) and they can be used to detect sentiment.

In [106]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/sentiment140.csv", header=None, usecols=[0, 5]) # no header in .csv

# Assign column names
df.columns = ['Sentiment', 'Text']

display(df.head())
print("Shape:", df.shape)

,Sentiment,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Shape: (1600000, 2)


### Sample

Get a `sample` from this huge dataset (otherwise won't work).

In [107]:
import random

# Sample
sample_size = 10_000
indices = random.sample(range(len(df)), sample_size)
df = df.iloc[indices]

display(df.head())
print("Shape:", df.shape)

,Sentiment,Text
1408295,4,@sara_tx I'm glad you and Cyrus made it home.
491156,0,mowing the yard...my favorite pass time
86743,0,Tryin to sleep. :/ Its hard.
16733,0,i feel i need a hug
432174,0,What to do for the last week of summer *hopefu...


Shape: (10000, 2)


### Split data

Split the dataset into training and testing `data sets`.

In [108]:
from sklearn.model_selection import train_test_split

X = df['Text']
y = df['Sentiment']
X1, X2, y1, y2 = train_test_split(X, y, test_size=0.2, random_state=0)

display(X1.head())
display(y1.head())

1557092    had a blast this weekend it was melissas bday,...
1184596                         @mileycyrus i voted for you 
1518980    @DivaWonderGirl OMJ! U hv 94 followers  I thin...
316149             @webholics same shit here...  #hosteurope
965394                                           @WereATeam 
Name: Text, dtype: object

1557092    4
1184596    4
1518980    4
316149     0
965394     4
Name: Sentiment, dtype: int64

### Vectorize

Convert text into `numerical` features.

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X1 = tfidf_vectorizer.fit_transform(X1)
X2 = tfidf_vectorizer.transform(X2)

print(X1.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### KNN Classifier

Train the KNN `classifier` using train (X1) dataset.

In [110]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

knn = KNeighborsClassifier()
knn.fit(X1, y1)

# Make predictions on the test set
y_pred = knn.predict(X2)

# Reports
print("Sentiment140 / samples =", sample_size)
print("Score on Train:", knn.score(X1, y1).round(2))
print("Score on Test:", knn.score(X2, y2).round(2))
print("Report:", classification_report(y2, y_pred))

Sentiment140 / samples = 10000
Score on Train: 0.8
Score on Test: 0.67
Report:               precision    recall  f1-score   support

           0       0.66      0.71      0.68       998
           4       0.69      0.63      0.66      1002

    accuracy                           0.67      2000
   macro avg       0.67      0.67      0.67      2000
weighted avg       0.67      0.67      0.67      2000



### Predictions

Make prediction for `reviews` (from .csv file).

In [111]:
# Load reviews
df_reviews = pd.read_csv('data/reviews.csv')

# Convert the sentiment labels
df_reviews['Sentiment'] = df_reviews['Sentiment'].map({'negative': 0, 'positive': 4})

X_unknown = df_reviews['Review']
y_unknown = df_reviews['Sentiment']

labels = {0: 'negative', 4: 'positive'}

# Convert using vectorizer
X_unknown = tfidf_vectorizer.transform(X_unknown)
y_pred_unknown = knn.predict(X_unknown)

print("Reviews (unknown):")
print("Unknown:\t", y_unknown.values)
print("Prediction:\t", y_pred_unknown)
print("Score on Reviews:", knn.score(X_unknown, y_unknown).round(2))

Reviews (unknown):
Unknown:	 [0 0 4 4 4 4 4 4 4 4 4 4 4 0 0 0 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 4 4 0]
Prediction:	 [0 0 0 0 4 0 0 0 0 0 4 4 4 0 0 4 0 0 4 0 0 4 0 0 0 0 4 0 0 0 0 0 0 0 0]
Score on Reviews: 0.57


### Random One

Prediction for `one random` review.

In [116]:
random_index = df_reviews.sample().index.item()

X_unknown_one = [df_reviews['Review'][random_index]]
y_unknown_one = df_reviews['Sentiment'][random_index]

labels = {0: 'negative', 4: 'positive'}

print("Review:\n", X_unknown_one[0])
print("Expected:", labels[y_unknown_one])

# Convert using vectorizer
X_unknown_one = tfidf_vectorizer.transform(X_unknown_one)
y_pred_one = knn.predict(X_unknown_one)

print("Prediction:", labels[y_unknown_pred[0]])

Review:
 Utter waste of money. As someone that has done a fair bit of coding in the past I wanted an introductory text to Python - something that would outline what could really be done with the programming language and provide an entry point to it. What a waste. The book is elementary on the one hand - and yet manages to miss a ton of required detail to actually do anything useful. Don't waste your time with it.
Expected: negative
Prediction: negative
